# Segmenting and Clustering Neighborhoods in Toronto

# Getting Data from Wikipedia Page

In [2]:
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge lxml --yes
import pandas as pd
import numpy as np
import requests
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


In [3]:
#Getting HTML of the website
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

# Getting table data from the soup object

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

postcode=[]
borough=[]
neighbourhood=[]
for tr in My_table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode.append([td.text.strip() for td in tds[:3]][0])
        borough.append([td.text.strip() for td in tds[:3]][1])
        neighbourhood.append([td.text.strip() for td in tds[:3]][2])
#My_table.a.get('title')

# Creating a Dataframe

In [5]:
df=pd.DataFrame()
df['Postcode']=postcode
df['Borough']=borough
df['Neighbourhood']=neighbourhood

# Removing rows with Borough is not assigned

In [6]:
df=df[~(df['Borough']=='Not assigned')]

In [7]:
df.loc[df['Borough'] == 'Queen\'s Park', ['Neighbourhood']] = 'Queen\'s Park'

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
df.shape

(212, 3)

# Grouping neighbourhoods with same postcode 

In [11]:

dfgrp=df.groupby('Postcode',as_index=False).agg(','.join)
dfgrp

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
7,M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
8,M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,"Scarborough,Scarborough","Birch Cliff,Cliffside West"


# Cleaning up Borough column

In [12]:
df_temp=dfgrp['Borough'].str.split(",", expand=True)
df_temp.rename(columns={0:'Borough'},inplace=True)

In [13]:
i = dfgrp.columns.get_loc('Borough')
dfgrp=pd.concat([dfgrp.iloc[:, :i], df_temp['Borough'], dfgrp.iloc[:, i+1:]], axis=1)

In [14]:
dfgrp.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
dfgrp.shape

(103, 3)

# Getting latitude and longitude values for each postcode

In [94]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALL

In [104]:
latitude=[]
longitude=[]

import geocoder # import geocoder
postal_code='M1E'
# initialize your variable to None
lat_lng_coords = None
for address in dfgrp['Postcode']:
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(address))
      lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [109]:
dfgrp['Latitude']=latitude
dfgrp['Longitude']=longitude

In [110]:
dfgrp

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696690,-79.260069


In [111]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [112]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfgrp['Latitude'], dfgrp['Longitude'], dfgrp['Borough'], dfgrp['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Filtering only Boroughs that have Toronto in the name

In [116]:
toronto_data = dfgrp[dfgrp['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676531,-79.295410
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667985,-79.314642
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090


In [120]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Explore using Foursquare API

In [122]:
CLIENT_ID = 'BM2PHLPJ4OZAPVXFO1PU41VQBVK4AXBNXHR320PZMNO0DHOJ' # your Foursquare ID
CLIENT_SECRET = 'T1D2QZJIWXZTWBGDJVT1HMYCKZPXXBTPO1WDLC0W2XKKBPXS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BM2PHLPJ4OZAPVXFO1PU41VQBVK4AXBNXHR320PZMNO0DHOJ
CLIENT_SECRET:T1D2QZJIWXZTWBGDJVT1HMYCKZPXXBTPO1WDLC0W2XKKBPXS


In [123]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [130]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names=toronto_data['Postcode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


In [131]:
print(toronto_venues.shape)
toronto_venues.head()

(1713, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676531,-79.29541,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,43.676531,-79.29541,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,43.676531,-79.29541,Starbucks,43.678798,-79.298045,Coffee Shop
3,M4E,43.676531,-79.29541,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.683262,-79.35512,Dairy Queen,43.684223,-79.357062,Fast Food Restaurant


# Analyzing each postcode

In [132]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,M4L,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.000000,0.040000,0.02,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.000000
4,M4N,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,M4S,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


# K means Clustering

In [205]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 3, 5, 5, 4, 5, 0, 5, 2, 5], dtype=int32)

In [206]:
toronto_grouped_final=toronto_grouped.copy()


In [207]:
toronto_grouped_final.insert(0,'Cluster',kmeans.labels_)

In [208]:
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(toronto_grouped_final.set_index('Postcode'), on='Postcode')

toronto_merged.tail() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,...,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.63941,-79.425436,5.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.037037,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
34,M6P,West Toronto,"High Park,The Junction South",43.65973,-79.462812,1.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
35,M6R,West Toronto,"Parkdale,Roncesvalles",43.64787,-79.449762,5.0,0.0,0.0,0.037736,0.0,...,0.0,0.0,0.000000,0.0,0.018868,0.0,0.0,0.0,0.0,0.0
36,M6S,West Toronto,"Runnymede,Swansea",43.64957,-79.475057,5.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.024390,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.385440,5.0,0.0,0.0,0.030000,0.0,...,0.0,0.0,0.010000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [209]:
toronto_merged=toronto_merged[~(toronto_merged['Cluster'].isnull())]

In [210]:
toronto_merged['Cluster']=toronto_merged['Cluster'].astype(int)

In [211]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters